# Day 32 - XGBoost, LightGBM, and OLS and NN

https://www.kaggle.com/aharless/xgboost-lightgbm-and-ols-and-nn

In [1]:
# 매개변수
FUDGE_FACTOR = 1.1200  # forecasts와 곱하기
XGB_WEIGHT = 0.6200
BASELINE_WEIGHT = 0.0100
OLS_WEIGHT = 0.0620
NN_WEIGHT = 0.0800
XGB1_WEIGHT = 0.8000  # 두 XGB 모델의 조합에서 첫번째의 가중치
BASELINE_PRED = 0.0115   # Oleg당 training 데이터의 평균 기준 baseline

In [4]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc
from sklearn.linear_model import LinearRegression
import random
import datetime as dt

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout, BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.layers.noise import GaussianDropout
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# Read in Raw Data

In [5]:
prop = pd.read_csv('../input/properties_2016.csv')
train = pd.read_csv('../input/train_2016_v2.csv')

c:\anaconda3\envs\study\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (22,32,34,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# LightGBM

## Process Data for LightGBM

In [7]:
for c, dtype in zip(prop.columns, prop.dtypes):
    if dtype == np.float64:
        prop[c] = prop[c].astype(np.float32)
        
df_train = train.merge(prop, how='left', on='parcelid')
df_train.fillna(df_train.median(), inplace=True)

x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 
                         'propertycountylandusecode', 'fireplacecnt', 'fireplaceflag'], axis=1)
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)

(90275, 53) (90275,)


In [8]:
train_columns = x_train.columns
for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)
    
del df_train
gc.collect()

x_train = x_train.values.astype(np.float32, copy=False)
d_train = lgb.Dataset(x_train, label=y_train)

## Run LightGBM

In [10]:
params = {}
params['max_bin'] = 10
params['learning_rate'] = 0.0021 # shrinkage_rate
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'l1'          # or 'mae'
params['sub_feature'] = 0.345    # feature_fraction (small values => use very different submodels)
params['bagging_fraction'] = 0.85 # sub_row
params['bagging_freq'] = 40
params['num_leaves'] = 512        # num_leaf
params['min_data'] = 500         # min_data_in_leaf
params['min_hessian'] = 0.05     # min_sum_hessian_in_leaf
params['verbose'] = 0
params['feature_fraction_seed'] = 2
params['bagging_seed'] = 3

np.random.seed(0)
random.seed(0)

In [11]:
# Fit LightGBM model
clf = lgb.train(params, d_train, 430)
del d_train
gc.collect()
del x_train
gc.collect()

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004932 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

0

In [12]:
# Prepare for LightGBM prediction
sample = pd.read_csv('../input/sample_submission.csv')
sample['parcelid'] = sample['ParcelId']

df_test = sample.merge(prop, on='parcelid', how='left')
del sample, prop
gc.collect()

x_test = df_test[train_columns]
del df_test
gc.collect()

0

In [13]:
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)

x_test = x_test.values.astype(np.float32, copy=False)

In [14]:
p_test = clf.predict(x_test)
del x_test
gc.collect()

pd.DataFrame(p_test).head()

,0
0,0.031132
1,0.033375
2,0.010257
3,0.008651
4,0.009660


# XGBoost

In [15]:
properties = pd.read_csv('../input/properties_2016.csv')

c:\anaconda3\envs\study\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (22,32,34,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Process Data for XGBoost

In [17]:
for c in properties.columns:
    properties[c] = properties[c].fillna(-1)
    if properties[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(properties[c].values))
        properties[c] = lbl.transform(list(properties[c].values))
        
train_df = train.merge(properties, how='left', on='parcelid')
x_train = train_df.drop(['parcelid','logerror','transactiondate'], axis=1)
x_test = properties.drop(['parcelid'], axis=1)

print('Shape train : {}\nShape test : {}'.format(x_train.shape, x_test.shape))

Shape train : (90275, 57)
Shape test : (2985217, 57)


## Drop out outliers

In [19]:
train_df = train_df[train_df.logerror > -0.4]
train_df = train_df[train_df.logerror < 0.419]
x_train = train_df.drop(['parcelid','logerror','transactiondate'], axis=1)
y_train = train_df['logerror'].values.astype(np.float32)
y_mean = np.mean(y_train)

print('After removing outliers :')
print('Shape train : {}\nShape test : {}'.format(x_train.shape, x_test.shape))

After removing outliers :
Shape train : (88528, 57)
Shape test : (2985217, 57)


## Run XGBoost

In [20]:
xgb_params = {
    'eta': 0.037,
    'max_depth': 5,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'lambda': 0.8,   
    'alpha': 0.4, 
    'base_score': y_mean,
    'silent': 1}

dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)

In [23]:
# train model
num_boost_rounds = 250
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)

# predict with XGBoost
xgb_pred1 = model.predict(dtest)
pd.DataFrame(xgb_pred1).head()

[20:56:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[20:56:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




,0
0,-0.030616
1,-0.028188
2,0.026397
3,0.063728
4,0.004398


## Run XGBoost Again

In [24]:
xgb_params = {
    'eta': 0.033,
    'max_depth': 6,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'base_score': y_mean,
    'silent': 1}
num_boost_rounds = 150
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)

xgb_pred2 = model.predict(dtest)
pd.DataFrame(xgb_pred2).head()

[20:57:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[20:57:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




,0
0,-0.091150
1,-0.034722
2,0.015816
3,0.075518
4,0.029908


## Combine XGBoost Results

In [25]:
xgb_pred = XGB1_WEIGHT * xgb_pred1 + (1-XGB1_WEIGHT) * xgb_pred2
pd.DataFrame(xgb_pred).head()

,0
0,-0.042723
1,-0.029495
2,0.024281
3,0.066086
4,0.009500


In [26]:
del train_df
del x_train
del x_test
del properties
del dtest
del dtrain
del xgb_pred1
del xgb_pred2
gc.collect()

273

# Neural Network

In [27]:
train = pd.read_csv('../input/train_2016_v2.csv', parse_dates=['transactiondate'])
prop = pd.read_csv('../input/properties_2016.csv')
sample = pd.read_csv('../input/sample_submission.csv')

c:\anaconda3\envs\study\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (22,32,34,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [28]:
# Fit label encoder on properties
for c in prop.columns:
    prop[c] = prop[c].fillna(-1)
    if prop[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(prop[c].values))
        prop[c] = lbl.transform(list(prop[c].values))

In [30]:
# Create training set
df_train = train.merge(prop, how='left', on='parcelid')
df_train['transactiondate'] = pd.to_datetime(df_train['transactiondate'])
df_train['transactiondate_year'] = df_train['transactiondate'].dt.year
df_train['transactiondate_month'] = df_train['transactiondate'].dt.month
df_train['transactiondate_quarter'] = df_train['transactiondate'].dt.quarter
df_train['transactiondate'] = df_train['transactiondate'].dt.day

df_train.fillna(-1.0)

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,transactiondate_year,transactiondate_month,transactiondate_quarter
0,11016594,0.0276,1,1.0,-1.0,-1.0,2.0,3.0,-1.0,4.0,...,360170.0,2015.0,237416.0,6735.88,0,-1.0,6.037107e+13,2016,1,1
1,14366692,-0.1684,1,-1.0,-1.0,-1.0,3.5,4.0,-1.0,-1.0,...,585529.0,2015.0,239071.0,10153.02,0,-1.0,-1.000000e+00,2016,1,1
2,12098116,-0.0040,1,1.0,-1.0,-1.0,3.0,2.0,-1.0,4.0,...,119906.0,2015.0,57912.0,11484.48,0,-1.0,6.037464e+13,2016,1,1
3,12643413,0.0218,2,1.0,-1.0,-1.0,2.0,2.0,-1.0,4.0,...,244880.0,2015.0,73362.0,3048.74,0,-1.0,6.037296e+13,2016,1,1
4,14432541,-0.0050,2,-1.0,-1.0,-1.0,2.5,4.0,-1.0,-1.0,...,434551.0,2015.0,264977.0,5488.96,0,-1.0,6.059042e+13,2016,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90270,10774160,-0.0356,30,1.0,-1.0,-1.0,1.0,1.0,-1.0,4.0,...,191000.0,2015.0,147200.0,2495.24,0,-1.0,6.037132e+13,2016,12,4
90271,12046695,0.0070,30,-1.0,-1.0,-1.0,3.0,3.0,-1.0,4.0,...,161111.0,2015.0,43218.0,1886.54,0,-1.0,6.037301e+13,2016,12,4
90272,12995401,-0.2679,30,-1.0,-1.0,-1.0,2.0,4.0,-1.0,7.0,...,38096.0,2015.0,16088.0,1925.70,1,14.0,6.037433e+13,2016,12,4
90273,11402105,0.0602,30,-1.0,-1.0,-1.0,2.0,2.0,-1.0,4.0,...,165869.0,2015.0,32878.0,2285.57,0,-1.0,6.037601e+13,2016,12,4


In [31]:
x_train = df_train.drop(['parcelid','logerror','transactiondate','propertyzoningdesc', 'propertycountylandusecode','fireplacecnt', 'fireplaceflag'], axis=1)
y_train = df_train['logerror']
y_mean = np.mean(y_train)

print(x_train.shape, y_train.shape)

train_columns = x_train.columns
for c in x_train.dtypes[x_train.dtypes==object].index.values:
    x_train[c] = (x_train[c]==True)

(90275, 56) (90275,)


In [32]:
# Create df_test
sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(prop, on='parcelid', how='left')

df_test["transactiondate"] = pd.to_datetime('2016-11-15') 
df_test["transactiondate_year"] = df_test["transactiondate"].dt.year
df_test["transactiondate_month"] = df_test["transactiondate"].dt.month
df_test['transactiondate_quarter'] = df_test['transactiondate'].dt.quarter
df_test["transactiondate"] = df_test["transactiondate"].dt.day     

x_test = df_test[train_columns]

print('Shape of x_test :', x_test.shape)

Shape of x_test : (2985217, 56)


In [33]:
# prepare x_test
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c]==True)

## Preprocessing

In [37]:
imputer = SimpleImputer()
imputer.fit(x_train.iloc[:,:])
x_train = imputer.transform(x_train.iloc[:,:])
imputer.fit(x_test.iloc[:,:])
x_test = imputer.transform(x_test.iloc[:,:])

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

len_x = int(x_train.shape[1])
print('len_x is :', len_x)

len_x is : 56


## Set up Neural Network

In [38]:
nn = Sequential()
nn.add(Dense(units=400, kernel_initializer='normal', input_dim=len_x))
nn.add(PReLU())           ### 가중치 초기화 : normal
nn.add(Dropout(.4))

nn.add(Dense(units=160, kernel_initializer='normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.6))

nn.add(Dense(units=64, kernel_initializer='normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.5))

nn.add(Dense(units=26, kernel_initializer='normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.6))

nn.add(Dense(1, kernel_initializer='normal'))
nn.compile(loss='mae', optimizer=Adam(lr=4e-3, decay=1e-4))

## Fit and Predict

In [39]:
nn.fit(np.array(x_train), np.array(y_train), batch_size=32, epochs=70, verbose=2)
y_pred_ann = nn.predict(x_test)
nn_pred = y_pred_ann.flatten()

print('Type of nn_pred is ', type(nn_pred))
print('Shape of nn_pred is ', nn_pred.shape)
print(pd.DataFrame(nn_pred).head())

Epoch 1/70
2822/2822 - 10s - loss: 0.0710
Epoch 2/70
2822/2822 - 8s - loss: 0.0682
Epoch 3/70
2822/2822 - 9s - loss: 0.0682
Epoch 4/70
2822/2822 - 9s - loss: 0.0681
Epoch 5/70
2822/2822 - 9s - loss: 0.0680
Epoch 6/70
2822/2822 - 8s - loss: 0.0680
Epoch 7/70
2822/2822 - 9s - loss: 0.0679
Epoch 8/70
2822/2822 - 9s - loss: 0.0679
Epoch 9/70
2822/2822 - 10s - loss: 0.0678
Epoch 10/70
2822/2822 - 10s - loss: 0.0678
Epoch 11/70
2822/2822 - 11s - loss: 0.0678
Epoch 12/70
2822/2822 - 14s - loss: 0.0677
Epoch 13/70
2822/2822 - 15s - loss: 0.0677
Epoch 14/70
2822/2822 - 13s - loss: 0.0677
Epoch 15/70
2822/2822 - 11s - loss: 0.0677
Epoch 16/70
2822/2822 - 10s - loss: 0.0676
Epoch 17/70
2822/2822 - 9s - loss: 0.0676
Epoch 18/70
2822/2822 - 8s - loss: 0.0676
Epoch 19/70
2822/2822 - 8s - loss: 0.0676
Epoch 20/70
2822/2822 - 8s - loss: 0.0676
Epoch 21/70
2822/2822 - 8s - loss: 0.0675
Epoch 22/70
2822/2822 - 9s - loss: 0.0676
Epoch 23/70
2822/2822 - 9s - loss: 0.0675
Epoch 24/70
2822/2822 - 10s - loss

In [42]:
# Cleanup
del train
del prop
del sample
del x_train
del x_test
del df_train
del df_test
del y_pred_ann
gc.collect()

4461

# OLS

## Processing data for OLS

In [54]:
np.random.seed(17)
random.seed(17)

train = pd.read_csv('../input/train_2016_v2.csv', parse_dates=['transactiondate'])
properties = pd.read_csv('../input/properties_2016.csv')
submission = pd.read_csv('../input/sample_submission.csv')
print(len(train), len(properties), len(submission))

c:\anaconda3\envs\study\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (22,32,34,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


90275 2985217 2985217


In [55]:
def get_features(df):
    df["transactiondate"] = pd.to_datetime(df["transactiondate"])
    df["transactiondate_year"] = df["transactiondate"].dt.year
    df["transactiondate_month"] = df["transactiondate"].dt.month
    df['transactiondate'] = df['transactiondate'].dt.quarter
    df = df.fillna(-1.0)
    return df

def MAE(y, ypred):
    return np.sum([abs(y[i]-ypred[i]) for i in range(len(y))]) / len(y)

In [57]:
import warnings
warnings.filterwarnings('ignore')

train = pd.merge(train, properties, how='left', on='parcelid')
y = train['logerror'].values
test = pd.merge(submission, properties, how='left', left_on='ParcelId', right_on='parcelid')
properties = []  # memory

exc = [train.columns[c] for c in range(len(train.columns)) if train.dtypes[c]=='O'] + ['logerror', 'parcelid']
col = [c for c in train.columns if c not in exc]

train = get_features(train[col])
test['transactiondate'] = '2016-01-01'
test = get_features(test[col])

## Fit OLS

In [59]:
reg = LinearRegression(n_jobs=-1)
reg.fit(train, y)
print(MAE(y, reg.predict(train)))

train = []  # memory
y = []  

test_dates = ['2016-10-01','2016-11-01','2016-12-01','2017-10-01','2017-11-01','2017-12-01']
test_columns = ['201610','201611','201612','201710','201711','201712']

0.0683700871188851


# Combine and Save

## Combine Predictions

In [61]:
lgb_weight = 1 - XGB_WEIGHT - BASELINE_WEIGHT - NN_WEIGHT - OLS_WEIGHT
lgb_weight0 = lgb_weight / (1-OLS_WEIGHT)
xgb_weight0 = XGB_WEIGHT / (1-OLS_WEIGHT)
baseline_weight0 = BASELINE_WEIGHT / (1-OLS_WEIGHT)
nn_weight0 = NN_WEIGHT / (1-OLS_WEIGHT)

pred0 = 0
pred0 += xgb_weight0 * xgb_pred
pred0 += baseline_weight0 * BASELINE_PRED
pred0 += lgb_weight0 * p_test
pred0 += nn_weight0 * nn_pred

pd.DataFrame(pred0).head()

,0
0,-0.021652
1,-0.012732
2,0.096834
3,0.060143
4,0.023488


In [65]:
for i in range(len(test_dates)):
    test['transactiondate'] = test_dates[i]
    pred = FUDGE_FACTOR * (OLS_WEIGHT * reg.predict(get_features(test)) + (1-OLS_WEIGHT)*pred0)
    submission[test_columns[i]] = [float(format(x,'.4f')) for x in pred]
    print('predict...', i)
    
submission.head()

predict... 0
predict... 1
predict... 2
predict... 3
predict... 4
predict... 5


,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,-0.0251,-0.0251,-0.0251,-0.0251,-0.0251,-0.0251
1,10759547,-0.0161,-0.0161,-0.0161,-0.0161,-0.0161,-0.0161
2,10843547,0.1468,0.1468,0.1468,0.1468,0.1468,0.1468
3,10859147,0.0659,0.0659,0.0659,0.0659,0.0659,0.0659
4,10879947,0.0268,0.0268,0.0268,0.0268,0.0268,0.0268


## Write the results

In [66]:
from datetime import datetime
submission.to_csv('sub{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False)